In [ ]:
import cirq
import os
from tests.test_BB_remove_T import *
import pandas as pd
from qramcircuits.toffoli_decomposition import ToffoliDecompType, ToffoliDecomposition
from mathematics.draper0406142 import CarryLookaheadAdder
import qramcircuits.bucket_brigade as bb
from utils.counting_utils import *
from utils.help_utils import *
import optimizers as qopt

# Test removal of T gates

In [ ]:
# provide arguments
decomp_ID = '1' # either 1, 2, or 3
percentage = 0.2 # in range [0.0, 1.0]
inplace = True # modification of circuit in place
repetitions = 10000 # repetitions of quantum circuit execution
save_dir = 'tests/results'

In [ ]:
# if save_dir doesn't exist create it
if os.path.exists(save_dir) is False:
    os.system(f"mkdir {save_dir}")

In [ ]:
# check removal of T gates for number of qubits from 2 to 3
for n_qubits in range(2, 4):
    print(f'-------- nqubits: {n_qubits} --------')
    # create decomposition scenario
    decomp_scenario = choose_decomposition(decomp_ID)

    # create dictionary to store results
    save_data = dict()
    save_data['input'] = []; save_data['output original'] = []; save_data['output modified'] = []

    # for every binary string of n_qubits -> check results
    for initial_state in create_binary_strings(n_qubits):
        print(f'-------- initial state: {initial_state} --------')

        # create BBcircuit with specified number of qubits
        bbcircuit = create_BB_circuit(n_qubits=n_qubits, decomp_scenario=decomp_scenario)

        
        # test removal of "percentage" of T gates
        bbcircuit, bbcircuit_modified, freq_origin, freq_mod = test_remove_T(bbcircuit,\
                                                                             initial_state=initial_state,\
                                                                             percentage=percentage,\
                                                                             inplace=inplace,\
                                                                             repetitions=repetitions)
        
        # gather results
        save_data['input'].append(initial_state)
        save_data['output original'].append(dict(freq_origin))
        save_data['output modified'].append(dict(freq_mod))
        
    #save results for specific qubit
    dataframe = pd.DataFrame(save_data)
    save_name = f'{save_dir}/nqubits_{n_qubits}_percentage_{percentage*100}.csv'
    dataframe.to_csv(save_name, index=False)  
    #verify_counts(bbcircuit, bbcircuit_modified, decomp_scenario)